# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [12]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
weather_file = '../Output/weather_data.csv'
weather_data = pd.read_csv(weather_file)
weather_data = weather_data.set_index('City')
weather_data = weather_data.drop(columns=['Unnamed: 0'])
weather_data.head()

,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City,,,,,,,,
hobart,-42.88,147.33,48.2,70,92,8.05,AU,1599760851
praia,14.92,-23.51,86.0,66,75,11.41,CV,1599761598
arraial do cabo,-22.97,-42.03,84.2,58,54,20.80,BR,1599760985
puerto ayora,-0.74,-90.35,75.2,60,40,19.46,EC,1599760886
kapaa,22.08,-159.32,80.6,69,40,13.87,US,1599761049


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [9]:
# Store latitude and longitude in locations
locations = weather_data[['Lat', 'Lng']]

# store humidity in variable
humidity = weather_data['Humidity'].astype(float)

# Plot heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=10, point_radius=1)

# Add layer
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [19]:
ideal_weather = weather_data.loc[(weather_data['Max Temp'] < 80) & (weather_data['Max Temp'] > 65) & (weather_data['Wind Speed'] < 10) & (weather_data['Cloudiness'] < 30) & (weather_data['Humidity'] < 50)]
# Drop any rows with null values
ideal_weather.dropna(inplace=True)
ideal_weather = ideal_weather.reset_index()
ideal_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,khash,28.22,61.22,70.88,21,0,8.93,IR,1599761612
1,buin,-33.73,-70.75,69.01,48,0,5.82,CL,1599761620
2,saint george,37.10,-113.58,72.00,27,1,3.36,US,1599760875
3,yulara,-25.24,130.99,68.00,28,0,1.12,AU,1599761635
4,gazli,40.13,63.45,69.57,23,0,9.55,UZ,1599761636
5,zambezi,-13.54,23.10,69.82,23,23,5.39,ZM,1599761653
6,mattawa,46.74,-119.90,75.20,40,1,2.68,US,1599761654
7,santiago,-33.46,-70.65,69.01,48,2,5.82,CL,1599761440
8,sayat,38.78,63.88,73.40,21,0,9.17,TM,1599761659
9,sheridan,44.83,-106.92,66.20,29,1,5.82,US,1599761659


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [20]:
hotel_df = ideal_weather[['City', "Lat", 'Lng']]
hotel_df['Hotel Name'] = ""
hotel_df

,City,Lat,Lng,Hotel Name
0,khash,28.22,61.22,
1,buin,-33.73,-70.75,
2,saint george,37.10,-113.58,
3,yulara,-25.24,130.99,
4,gazli,40.13,63.45,
5,zambezi,-13.54,23.10,
6,mattawa,46.74,-119.90,
7,santiago,-33.46,-70.65,
8,sayat,38.78,63.88,
9,sheridan,44.83,-106.92,


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
